In [17]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)


### State transition and emission matrices mined from Petri net model

In [61]:
W = [
    [0., 1., 0., 0., 0., 0., 0.],
    [0., 0., 0.5, 0.5, 0., 0., 0.],
    [0., 0., 0., 0., 1., 0., 0.],
    [0., 0., 0., 0., 1., 0., 0.],
    [0., 0., 0., 0., 0., 1., 0.],
    [0., 0.5, 0., 0., 0., 0., 0.5],
    [0., 0., 0., 0., 0., 0., 0.]
]

V = [
    [1., 0., 0., 0., 0., 0., 0.],
    [0., 0.25, 0., 0.5, 0., 0., 0.],
    [0., 0.25, 0., 0.5, 0., 0., 0.],
    [0., 0.5, 1.0, 0., 0., 0., 0.],
    [0., 0., 0., 0., 1., 0., 0.],
    [0., 0., 0., 0., 0., 0.5, 0.],
    [0., 0., 0., 0., 0., 0.5, 0.]
]

C = [
    [1., 0., 0., 0., 0., 0., 0.],
    [0., 1., 0., 1., 0., 0., 0.],
    [0., 1., 0., 1., 0., 0., 0.],
    [0., 1., 1., 0., 0., 0., 0.],
    [0., 0., 0., 0., 1., 0., 0.],
    [0., 0., 0., 0., 0., 1., 0.],
    [0., 0., 0., 0., 0., 1., 0.]
]

W = np.asarray(W)

V = np.asarray(V)
C = np.asarray(C)

# W and V matrices of deviating behavior
W_d = np.ones(W.shape) * 0.01
V_d = np.ones(V.shape) * 0.01

### Non-fitting case: <a, b, c, d, e, g> (log move)

#### Initial distribution $\pi$

In [53]:
init = [1., 0., 0., 0., 0., 0., 0.]
init = np.reshape(init, [1, 7])

#### At execution of < a >
$\alpha_j(1) = P(X_1 = a, Z_1 = j) = \pi_j \times v_j(a) = \pi_j \times [c(\pi, 1_a) \times V_{a, j} + (1 - c(\pi, 1_a)) V^d_{a, j}]$



In [54]:
c_1 = np.dot(init, C[0,:])
alpha_1 = init * (c_1 * V[0,:])

# re-estimate conformance
c_1_reest = np.dot(alpha_1, C[0,:])

print('P(X_1 = <a>, Z_1 = z_1): {}'.format(alpha_1))
print('Re-estimated conformance of execution "a": {}'.format(c_1_reest))

P(X_1 = <a>, Z_1 = z_1): [[ 1.  0.  0.  0.  0.  0.  0.]]
Re-estimated conformance of execution "a": [ 1.]


In [62]:
def conf(z, a):
    """
    Computes conformance given a state estimation and observed activity.
    """
    if (not np.isclose(z.sum(), [1.])):
        raise ValueError('State z: {} does not sum to 1: {}.'.format(z, z.sum()))
    
    ind = ord(a) - ord('a')    
    return np.dot(z, C[ind,:])


def v(z, a_k):
    """
    Computes P(x is a_k at time t | z at time t)
    
    z: state estimation at time t
    a_k: the observed activity
    """
    if (z.shape[1] != V.shape[1]):
        raise ValueError('Invalid state length: {}'.format(z))
    
    w_z = z.sum()
    weighted = z / w_z if w_z > 0 else z
    c = conf(weighted, a_k) if w_z > 0 else 0 # need to check for state estimate being 0. to avoid division by zero
    ind = ord(a_k) - ord('a')
    print('Conformance between state and observation at time t before observation adjustment: {}'.format(c))
    return c * V[ind,:] + (1 - c) * V_d[ind,:]
    

def w(z, a_k):
    """
    Computes P(z at time t + 1 | z at time t, x is a_k at time t)
    """
    if (z.shape[1] != W.shape[1]):
        raise ValueError('Invalid state length: {}'.format(z))
    
    w_z = np.sum(z)
    weighted = z / w_z if w_z > 0 else z
    c = conf(weighted, a_k) if w_z > 0 else 0 # need to check for state estimate being 0. to avoid division by zero
    return c * W + (1 - c) * W_d

    
def alpha(a_t, a_t1=None, prev_alpha=None, init=None):
    """
    a_t: activity executed at time t
    a_t1: activity executed at time t - 1
    prev_alpha: alpha from t - 1
    """
    if prev_alpha is None:
        # base case
        return init * v(init, a_t)
    
    est_cur_state = (w(prev_alpha, a_t1).T * prev_alpha).sum(axis=1).reshape([1, W.shape[1]])
    print('   State estimate of time t before observation at time t: {}'.format(est_cur_state))
    print('W. state estimate of time t before observation at time t: {}'.format(est_cur_state / est_cur_state.sum()))
    obs_likelihood = v(est_cur_state, a_t)
    print('Likelihood of observation at states at time t: {}'.format(obs_likelihood))
    return obs_likelihood * est_cur_state

#### At execution of < a, b >
$\alpha_j(2) = P(X_{1:2} = <a, b>, Z_2 = j) = \sum_{i \in Z} v_j(b) w_{ij}(a) \alpha_i(1)$


In [63]:
this_act = 'b'
prev_act = 'a'
prev_alpha = alpha_1

alpha_2 = alpha(this_act, prev_act, prev_alpha)

w_alpha_2 = alpha_2.sum()
weighted_alpha_2 = alpha_2 / w_alpha_2 if w_alpha_2 > 0 else alpha_2
c_2_reest = conf(weighted_alpha_2, this_act) if w_alpha_2 > 0 else 0 # need to check for state estimate being 0. to avoid division by zero

print('\n   P(X_1:2 = <{}, {}>, Z_2 = z_2): {}'.format(prev_act, this_act, alpha_2))
print('W. P(X_1:2 = <{}, {}>, Z_2 = z_2): {}'.format(prev_act, this_act, weighted_alpha_2))
print('Re-estimated conformance of execution "{}": {}'.format(this_act, c_2_reest))

   State estimate of time t before observation at time t: [[ 0.  1.  0.  0.  0.  0.  0.]]
W. state estimate of time t before observation at time t: [[ 0.  1.  0.  0.  0.  0.  0.]]
Conformance between state and observation at time t before observation adjustment: [ 1.]
Likelihood of observation at states at time t: [ 0.    0.25  0.    0.5   0.    0.    0.  ]

   P(X_1:2 = <a, b>, Z_2 = z_2): [[ 0.    0.25  0.    0.    0.    0.    0.  ]]
W. P(X_1:2 = <a, b>, Z_2 = z_2): [[ 0.  1.  0.  0.  0.  0.  0.]]
Re-estimated conformance of execution "b": [ 1.]


#### At execution of < a, b, c >
$\alpha_j(3) = P(X_{1:3} = <a, b, c>, Z_3 = j) = \sum_{i \in Z} v_j(c) w_{ij}(b) \alpha_i(2)$


In [64]:
this_act = 'c'
prev_act = 'b'
prev_alpha = alpha_2

alpha_3 = alpha(this_act, prev_act, prev_alpha)

w_alpha_3 = alpha_3.sum()
weighted_alpha_3 = alpha_3 / w_alpha_3 if w_alpha_3 > 0 else alpha_3
c_3_reest = conf(weighted_alpha_3, this_act) if w_alpha_3 > 0 else 0

print('\n   P(X_1:3 = <a, {}, {}>, Z_3 = z_3): {}'.format(prev_act, this_act, alpha_3))
print('W. P(X_1:3 = <a, {}, {}>, Z_3 = z_3): {}'.format(prev_act, this_act, weighted_alpha_3))
print('Re-estimated conformance of execution "{}": {}'.format(this_act, c_3_reest))

   State estimate of time t before observation at time t: [[ 0.     0.     0.125  0.125  0.     0.     0.   ]]
W. state estimate of time t before observation at time t: [[ 0.   0.   0.5  0.5  0.   0.   0. ]]
Conformance between state and observation at time t before observation adjustment: [ 0.5]
Likelihood of observation at states at time t: [ 0.005  0.13   0.005  0.255  0.005  0.005  0.005]

   P(X_1:3 = <a, b, c>, Z_3 = z_3): [[ 0.       0.       0.00063  0.03188  0.       0.       0.     ]]
W. P(X_1:3 = <a, b, c>, Z_3 = z_3): [[ 0.       0.       0.01923  0.98077  0.       0.       0.     ]]
Re-estimated conformance of execution "c": [ 0.98077]


#### At execution of < a, b, c, d >
$\alpha_j(4) = P(X_{1:4} = <a, b, c, d>, Z_4 = j) = \sum_{i \in Z} v_j(d) w_{ij}(c) \alpha_i(3)$


In [65]:
this_act = 'd'
prev_act = 'c'
prev_alpha = alpha_3

alpha_4 = alpha(this_act, prev_act, prev_alpha)

w_alpha_4 = alpha_4.sum()
weighted_alpha_4 = alpha_4 / w_alpha_4 if w_alpha_4 > 0 else alpha_4
c_4_reest = conf(weighted_alpha_4, this_act) if w_alpha_4 > 0 else 0

print('\n   P(X_1:4 = <a, b, {}, {}>, Z_4 = z_4): {}'.format(prev_act, this_act, alpha_4))
print('W. P(X_1:4 = <a, b, {}, {}>, Z_4 = z_4): {}'.format(prev_act, this_act, weighted_alpha_4))
print('Re-estimated conformance of execution "{}": {}'.format(this_act, c_4_reest))

   State estimate of time t before observation at time t: [[ 0.00001  0.00001  0.00001  0.00001  0.03188  0.00001  0.00001]]
W. state estimate of time t before observation at time t: [[ 0.0002   0.0002   0.0002   0.0002   0.99883  0.0002   0.0002 ]]
Conformance between state and observation at time t before observation adjustment: [ 0.00039]
Likelihood of observation at states at time t: [ 0.01     0.01019  0.01039  0.01     0.01     0.01     0.01   ]

   P(X_1:4 = <a, b, c, d>, Z_4 = z_4): [[ 0.       0.       0.       0.       0.00032  0.       0.     ]]
W. P(X_1:4 = <a, b, c, d>, Z_4 = z_4): [[ 0.0002   0.0002   0.0002   0.0002   0.99881  0.0002   0.0002 ]]
Re-estimated conformance of execution "d": [ 0.0004]


#### At execution of < a, b, c, d, e >
$\alpha_j(5) = P(X_{1:5} = <a, b, c, d, e>, Z_5 = j) = \sum_{i \in Z} v_j(e) w_{ij}(d) \alpha_i(4)$


In [66]:
this_act = 'e'
prev_act = 'd'
prev_alpha = alpha_4

alpha_5 = alpha(this_act, prev_act, prev_alpha)

w_alpha_5 = alpha_5.sum()
weighted_alpha_5 = alpha_5 / w_alpha_5 if w_alpha_5 > 0 else alpha_5
c_5_reest = conf(weighted_alpha_5, this_act) if w_alpha_5 > 0 else 0

print('\n   P(X_1:5 = <a, b, c, {}, {}>, Z_5 = z_5): {}'.format(prev_act, this_act, alpha_5))
print('W. P(X_1:5 = <a, b, c, {}, {}>, Z_5 = z_5): {}'.format(prev_act, this_act, weighted_alpha_5))
print('Re-estimated conformance of execution "{}": {}'.format(this_act, c_5_reest))

   State estimate of time t before observation at time t: [[ 0.  0.  0.  0.  0.  0.  0.]]
W. state estimate of time t before observation at time t: [[ 0.14204  0.14204  0.14204  0.14204  0.14204  0.14776  0.14204]]
Conformance between state and observation at time t before observation adjustment: [ 0.14204]
Likelihood of observation at states at time t: [ 0.00858  0.00858  0.00858  0.00858  0.15062  0.00858  0.00858]

   P(X_1:5 = <a, b, c, d, e>, Z_5 = z_5): [[ 0.  0.  0.  0.  0.  0.  0.]]
W. P(X_1:5 = <a, b, c, d, e>, Z_5 = z_5): [[ 0.04238  0.04238  0.04238  0.04238  0.74401  0.04409  0.04238]]
Re-estimated conformance of execution "e": [ 0.74401]


#### At execution of < a, b, c, d, e, g >
$\alpha_j(6) = P(X_{1:5} = <a, b, c, d, e, g>, Z_6 = j) = \sum_{i \in Z} v_j(g) w_{ij}(e) \alpha_i(5)$


In [70]:
this_act = 'g'
prev_act = 'e'
prev_alpha = alpha_5

alpha_6 = alpha(this_act, prev_act, prev_alpha)

w_alpha_6 = alpha_6.sum()
weighted_alpha_6 = alpha_6 / w_alpha_6 if w_alpha_6 > 0 else alpha_6
c_6_reest = conf(weighted_alpha_6, this_act) if w_alpha_6 > 0 else 0

print('\n   P(X_1:6 = <a, b, c, d, {}, {}>, Z_6 = z_6): {}'.format(prev_act, this_act, alpha_6))
print('W. P(X_1:6 = <a, b, c, d, {}, {}>, Z_6 = z_6): {}'.format(prev_act, this_act, weighted_alpha_6))
print('Re-estimated conformance of execution "{}": {}'.format(this_act, c_6_reest))

   State estimate of time t before observation at time t: [[ 0.  0.  0.  0.  0.  0.  0.]]
W. state estimate of time t before observation at time t: [[ 0.0035   0.06913  0.02509  0.02509  0.08984  0.76139  0.02596]]
Conformance between state and observation at time t before observation adjustment: [ 0.76139]
Likelihood of observation at states at time t: [ 0.00239  0.00239  0.00239  0.00239  0.00239  0.38308  0.00239]

   P(X_1:6 = <a, b, c, d, e, g>, Z_6 = z_6): [[ 0.  0.  0.  0.  0.  0.  0.]]
W. P(X_1:6 = <a, b, c, d, e, g>, Z_6 = z_6): [[ 0.00003  0.00056  0.0002   0.0002   0.00073  0.99805  0.00021]]
Re-estimated conformance of execution "g": [ 0.99805]
